<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Breast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

In [0]:
train_data_dir = '/content/drive/My Drive/dataset'

In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.3)
batch_size = 32
img_width, img_height = 220, 220
# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 3297 images belonging to 2 classes.
Found 1412 images belonging to 2 classes.


In [0]:
epochs = 10
train_samples = 3297
validation_samples = 1412

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam 
#import pickle
import time

model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(220,220,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
'''
model.add(Conv2D(64, (5, 5), input_shape=(736,459,3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(128, (5, 5), input_shape=(736,459,3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
'''
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(2))
model.add(Activation('softmax'))

#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 216, 216, 32)      2432      
_________________________________________________________________
activation (Activation)      (None, 216, 216, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 108, 108, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 64)      51264     
_________________________________________________________________
activation_1 (Activation)    (None, 104, 104, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 52, 64)        0         
_________________________________________

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Breast.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
16/32 [==============>...............] - ETA: 5:04 - loss: 0.9968 - acc: 0.6836

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-4ad6745c4738>", line 16, in <module>
    verbose=1)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training.py", line 1296, in fit_generator
    steps_name='steps_per_epoch')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training_generator.py", line 221, in model_iteration
    batch_data = _get_next_batch(generator)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training_generator.py", line 363, in _get_next_batch
    generator_output = next(generator)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/utils/data_utils.py", line 783, in get
    inputs = self.queue.get(block=True).get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", 

KeyboardInterrupt: ignored

In [0]:
model.save("Breast.model")

In [0]:
import cv2
import tensorflow as tf
import math
#from google.colab.patches import cv2_imshow

CATEGORIES = ["cancer", "normal"]


def prepare(filepath):
    #IMG_SIZE = 220 # 50 in txt-based
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (736,459))

    return new_array.reshape(-1, 736,459, 3)


model = tf.keras.models.load_model("Breast.h5")

prediction = model.predict([prepare('/content/drive/My Drive/dataset/NORMAL/normal_04-A_0475_1.LEFT_MLO.LJPEG.1.jpg')])
pred_percent = math.floor(prediction[0][0]*100)
print(pred_percent,"%")
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])
'''
if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(prediction)
print(result*100)
print(CATEGORIES[int(result)])
'''

In [0]:
normal = 2342
cancer = 2367

if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(result)

0.50929433


In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen1 = ImageDataGenerator(rescale=1. / 255)
batch_size = 32
train_data_dir1 = '/content/drive/My Drive/dataset1/' 
# Retrieve images and their classes for train and validation sets
train_generator1 = datagen1.flow_from_directory(
        train_data_dir1,
        target_size=(224, 224),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

Found 2367 images belonging to 1 classes.


In [0]:
import os
import glob
from PIL import Image

image_list = []
i = 0
for filename in glob.glob('/content/drive/My Drive/dataset/CANCER/*.jpg'): #assuming jpg
    im=Image.open(filename)
    image_list.append(im)
    print(i)
    i +=1
len(image_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
from PIL import Image

im = Image.open('/content/drive/My Drive/dataset/CANCER/cancer_01-B_3003_1.LEFT_MLO.LJPEG.1.jpg')
im.save('/content/drive/My Drive/PNG/cancer/cancer_01-B_3003_1.LEFT_MLO.LJPEG.1.png')